In [21]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
data=pd.read_csv(r"C:\Users\vaibh\Downloads\Dataset .csv")

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [24]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Restaurant ID,9551.0,9.051128e+06,8.791521e+06,53.000000,301962.500000,6.004089e+06,1.835229e+07,1.850065e+07
Country Code,9551.0,1.836562e+01,5.675055e+01,1.000000,1.000000,1.000000e+00,1.000000e+00,2.160000e+02
Longitude,9551.0,6.412657e+01,4.146706e+01,-157.948486,77.081343,7.719196e+01,7.728201e+01,1.748321e+02
Latitude,9551.0,2.585438e+01,1.100794e+01,-41.330428,28.478713,2.857047e+01,2.864276e+01,5.597698e+01
Average Cost for two,9551.0,1.199211e+03,1.612118e+04,0.000000,250.000000,4.000000e+02,7.000000e+02,8.000000e+05
Price range,9551.0,1.804837e+00,9.056088e-01,1.000000,1.000000,2.000000e+00,2.000000e+00,4.000000e+00
Aggregate rating,9551.0,2.666370e+00,1.516378e+00,0.000000,2.500000,3.200000e+00,3.700000e+00,4.900000e+00
Votes,9551.0,1.569097e+02,4.301691e+02,0.000000,5.000000,3.100000e+01,1.310000e+02,1.093400e+04


In [25]:
# Preprocessing
data.dropna(subset=['Cuisines'], inplace=True)  # Drop rows with missing cuisines

In [26]:
data['Cuisines'] = data['Cuisines'].str.lower()  # Normalize cuisines to lowercase

In [27]:
# Combine features for recommendation
data['Features'] = data['Cuisines'] + ' ' + data['City'].str.lower()

In [28]:
# Convert text data into feature vectors
vectorizer = CountVectorizer()
feature_matrix = vectorizer.fit_transform(data['Features'])


In [29]:
# Compute similarity between restaurants
similarity_matrix = cosine_similarity(feature_matrix)

In [31]:
# Recommendation function
def recommend_restaurants(preferred_cuisine, city, top_n=5):
    preferred_features = preferred_cuisine.lower() + ' ' + city.lower()
    preferred_vector = vectorizer.transform([preferred_features])
    
    # Compute similarity with all restaurants
    similarities = cosine_similarity(preferred_vector, feature_matrix).flatten()
    
    # Get indices of top matching restaurants
    top_indices = similarities.argsort()[-top_n:][::-1]
    
    if len(top_indices) == 0:
        return "No recommendations found."

    # Fetch recommended restaurants
    recommendations = data.iloc[top_indices][['Restaurant Name', 'Cuisines', 'City', 'Aggregate rating']]
    return recommendations


In [40]:
# Example usage
user_cuisine = 'Japanese'
user_city = 'Makati City'
recommendations = recommend_restaurants(user_cuisine, user_city)


print("Top Recommended Restaurants:")
print(recommendations)

Top Recommended Restaurants:
                   Restaurant Name                    Cuisines  \
1                 Izakaya Kikufuji                    japanese   
0                 Le Petit Souffle  french, japanese, desserts   
4                      Sambo Kojin            japanese, korean   
480   Fuji Bay Japanese Restaurant             japanese, sushi   
9334                      wagamama             japanese, asian   

                  City  Aggregate rating  
1          Makati City               4.5  
0          Makati City               4.8  
4     Mandaluyong City               4.8  
480         Sioux City               3.7  
9334   Wellington City               3.7  
